In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import reduce_mean
from tensorflow.keras.optimizers import Adam
from tensorflow.nn import sigmoid_cross_entropy_with_logits as loss
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense
#from tensorflow.layers import dense, batch_normalization, conv2d_transpose, conv2d

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sarikamohanraj","key":"c1c8dc2c306071edb6410141f88029ab"}'}

In [ ]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/
!kaggle datasets download -d arnaud58/flickrfaceshq-dataset-ffhq



mkdir: cannot create directory ‘/root/.kaggle’: File exists
flickrfaceshq-dataset-ffhq.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile

# Replace 'path_to_downloaded_zip_file.zip' with the actual path to your downloaded zip file
with zipfile.ZipFile('flickrfaceshq-dataset-ffhq.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/GenAI lab/ex3/data')


In [ ]:
image_ids = glob.glob('/content/drive/MyDrive/GenAI lab/ex3/data/*')

crop = (30, 55, 150, 175)
images = [np.array((Image.open(i).crop(crop)).resize((64,64))) for i in image_ids]

for i in range(len(images)):
    images[i] = ((images[i] - images[i].min())/(255 - images[i].min()))
    images[i] = images[i]*2-1

images = np.array(images)

In [ ]:
def inputs(real_dim, noise_dim):

    inputs_real = tf.placeholder(tf.float32, (None, *real_dim), name='input_real')
    inputs_noise = tf.placeholder(tf.float32, (None, noise_dim), name='input_noise')

    return inputs_real, inputs_noise

In [ ]:
def generator(noise, reuse=False, alpha=0.2, training=True):

    with tf.variable_scope('generator', reuse=reuse):

        x = dense(noise, 4*4*512)
        x = tf.reshape(x, (-1, 4, 4, 512))
        x = batch_normalization(x, training=training)
        x = tf.maximum(alpha*x, x)

        x = conv2d_transpose(x, 256, 5, 2, padding='same')
        x = batch_normalization(x, training=training)
        x = tf.maximum(alpha*x, x)

        x = conv2d_transpose(x, 128, 5, 2, padding='same')
        x = batch_normalization(x, training=training)
        x = tf.maximum(alpha*x, x)

        x = conv2d_transpose(x, 64, 5, 2, padding='same')
        x = batch_normalization(x, training=training)
        x = tf.maximum(alpha*x, x)

        logits = conv2d_transpose(x, 3, 5, 2, padding='same')
        out = tf.tanh(logits)

        return out, logits

In [ ]:
def discriminator(x, reuse=False, alpha=0.2, training=True):

    with tf.variable_scope('discriminator', reuse=reuse):

        x = conv2d(x, 32, 5, 2, padding='same')
        x = tf.maximum(alpha*x, x)

        x = conv2d(x, 64, 5, 2, padding='same')
        x = batch_normalization(x, training=training)
        x = tf.maximum(alpha*x, x)

        x = conv2d(x, 128, 5, 2, padding='same')
        x = batch_normalization(x, training=training)
        x = tf.maximum(alpha*x, x)

        x = conv2d(x, 256, 5, 2, padding='same')
        x = batch_normalization(x, training=training)
        x = tf.maximum(alpha*x, x)

        flatten = tf.reshape(x, (-1, 4*4*256))
        logits = dense(flatten, 1)
        out = tf.sigmoid(logits)

        return out, logits

In [ ]:
# hyperparameters

beta1 = 0.5
alpha = 0.2
smooth = 0.9
noise_size = 200
learning_rate = 0.0002
input_shape = (64,64,3)

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0


In [ ]:
# building the graph

tf.reset_default_graph()

input_real, input_noise = inputs(input_shape, noise_size)

gen_noise, gen_logits = generator(input_noise)

dis_out_real, dis_logits_real = discriminator(input_real)
dis_out_fake, dis_logits_fake = discriminator(gen_noise, reuse=True)

AttributeError: ignored

In [ ]:
# defining losses

shape = dis_logits_real

dis_loss_real = reduce_mean(loss(logits=dis_logits_real, labels=tf.ones_like(shape*smooth)))

dis_loss_fake = reduce_mean(loss(logits=dis_logits_fake, labels=tf.zeros_like(shape)))

gen_loss = reduce_mean(loss(logits=dis_logits_fake, labels=tf.ones_like(shape*smooth)))

dis_loss = dis_loss_real + dis_loss_fake

In [ ]:
# defining optimizers

total_vars = tf.trainable_variables()

dis_vars = [var for var in total_vars if var.name[0] == 'd']
gen_vars = [var for var in total_vars if var.name[0] == 'g']

dis_opt = adam(learning_rate=learning_rate, beta1=beta1).minimize(dis_loss, var_list=dis_vars)
gen_opt = adam(learning_rate=learning_rate, beta1=beta1).minimize(gen_loss, var_list=gen_vars)

In [ ]:
def view_samples(epoch, samples, nrows, ncols, figsize=(5,5)):

    fig, axes = plt.subplots(figsize=figsize, nrows=nrows, ncols=ncols)

    for ax, img in zip(axes.flatten(), samples[epoch]):

        ax.axis('off')
        # img = (((img+1)/2)*255).astype(np.uint8)
        img = ((img - img.min())*255 / (img.max() - img.min())).astype(np.uint8)
        im = ax.imshow(img)

    plt.subplots_adjust(wspace=0, hspace=0)
    return fig, axes

In [ ]:
batch_size = 128
epochs = 10
iters = len(image_ids)//batch_size
saver = tf.train.Saver(var_list = gen_vars)

with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    for e in range(epochs):

        for i in range(iters-1):

            batch_images = images[i*batch_size:(i+1)*batch_size]
            batch_noise = np.random.uniform(-1, 1, size=(batch_size, noise_size))

            sess.run(dis_opt, feed_dict={input_real: batch_images, input_noise: batch_noise})
            sess.run(gen_opt, feed_dict={input_real: batch_images, input_noise: batch_noise})

            if i%50 == 0:
                print("Epoch {}/{}...".format(e+1, epochs), "Batch No {}/{}".format(i+1, iters))

        loss_dis = sess.run(dis_loss, {input_noise: batch_noise, input_real: batch_images})
        loss_gen = gen_loss.eval({input_real: batch_images, input_noise: batch_noise})

        print("Epoch {}/{}...".format(e+1, epochs),"Discriminator Loss: {:.4f}...".format(loss_dis),
              "Generator Loss: {:.4f}".format(loss_gen))

        sample_noise = np.random.uniform(-1, 1, size=(8, noise_size))
        gen_samples = sess.run(generator(input_noise, reuse=True, alpha=alpha),
                               feed_dict={input_noise: sample_noise})

        view_samples(-1, gen_samples, 2, 4, (10,5))
        plt.show()
        saver.save(sess, './checkpoints/generator.ckpt')

AttributeError: ignored